In [47]:
import numpy as np
import pandas as pd
from transaction_dates import *
from imiona import *
from collections import Counter

# Pracownicy

In [72]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["szef", "sprzedawca", "mechanik", "pomocnik mechanika", "blacharz", 
                      "lakiernik", "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 4), rind(2, 4), rind(1, 3), 1, 
                        1, rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates("2014-01-01", "2018-01-01", np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45), (40, 55),
              (40, 55), (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "data_urodzenia", "pesel",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca[zł/h]"])
for (i, job) in enumerate(jobs_list):
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, birth_date, pesel, city, address, phone, email, employment_date, None, job, wage]

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [73]:
employees_df

,id,imię,nazwisko,data_urodzenia,pesel,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca[zł/h]
0,1,Małgorzata,Martowska,25/07/1985,85072546326,Wrocław,ul. Lawendowa 65/10,48040765239,malgorzata.martowska@kuba.pl,2014-01-01,None,szef,73.8
1,2,Mariusz,Skalski-Cichoń,16/05/1983,83051633353,Wrocław,ul. Fryderyka Chopina 88/21,48409680555,mariusz.skalski-cichon@kuba.pl,2014-01-01,None,sprzedawca,50.7
2,3,Amelia,Meger,26/07/1974,74072656788,Wrocław,ul. Jana Długosza 28/49,48253414428,amelia.meger@kuba.pl,2014-01-01,None,sprzedawca,52.1
3,4,Magdalena,Woźniak,04/12/1990,90120442107,Wrocław,ul. Jana Pawła II 98/18,48646755377,magdalena.wozniak@kuba.pl,2014-01-01,None,mechanik,58.7
9,5,Marcin,Gorski,20/03/1961,61032047254,Wrocław,ul. Górka 30/48,48334185596,marcin.gorski@kuba.pl,2014-01-01,None,diagnosta samochodowy,57.6
4,6,Marcin,Wróblewski,05/07/1963,63070559018,Bydgoszcz,ul. Cicha 6/3,48613834241,marcin.wroblewski@kuba.pl,2015-07-31,None,mechanik,50.2
5,7,Kinga,Król,30/08/1991,91083018646,Polkowice,ul. Słowicza 15/18,48168671461,kinga.krol@kuba.pl,2015-10-02,None,pomocnik mechanika,44.2
6,8,Valentin,Sar,20/04/1968,68042010074,Wrocław,ul. Kręta 13/36,48679245749,valentin.sar@kuba.pl,2015-10-14,None,pomocnik mechanika,41.2
7,9,Agnieszka,Jajko,18/05/1971,71051841966,Opole,ul. Goździkowa 83,48442325723,agnieszka.jajko@kuba.pl,2015-10-30,None,blacharz,54.0
8,10,Marta,Lahutsina,08/10/1991,91100843385,Września,ul. Wąska 43/46,48066970589,marta.lahutsina@kuba.pl,2016-04-10,None,lakiernik,44.3


# Transakcje

In [50]:
no_transactions = 2000
begining_buy = 20

dates = random_transaction_dates("2014-01-01", "2020-12-31", no_transactions)

In [51]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [52]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([len(clients_cars) , len(our_cars)*50, len(clients_cars)/5])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            jp += 1
            jc += 1

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            jp += 1
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1

In [53]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [54]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

# klienci

In [55]:
year_lims = [1970, 2010]
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "data_urodzenia", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])
unique_clients = np.unique(client_ids)
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        pesel, birth_date = random_pesel_date_man(year_lims)
        sex = "M"
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, birth_date[-4:])
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, birth_date, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        pesel, birth_date = random_pesel_date_woman(year_lims)
        sex = "F"
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, birth_date[-4:])
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, birth_date, sex, pesel, city, address, loyalty_card, phone, email]

In [56]:
clients_df

,id,imię,nazwisko,data_urodzenia,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Jakub,Dobrowolski,15/08/1972,M,72081528294,Wrocław,ul. Jabłoniowa 68/31,False,48751355370,k4y0vayjasz@opayq.com
1,2,Aleksandra,Kovalchuk,28/01/1988,F,88012874589,Wrocław,ul. Smutna 16,False,48351584939,kovalchuk5@interia.pl
2,3,Piotr,Wasilewski,18/12/1975,M,75121818078,Wrocław,ul. Klimontowska 11/8,False,48320687650,pwasile1975@gmail.com
3,4,Katarzyna,Głowacka,20/11/1979,F,79112065629,Wrocław,ul. Portowa 40/50,False,48603395395,yizai60vyl54@opayq.com
4,5,Stanisława,Podgórska,22/07/1992,F,92072203724,Wrocław,ul. Spokojna 14/24,False,48139425547,podgorska19927@gmail.com
...,...,...,...,...,...,...,...,...,...,...,...
1514,1515,Stanisław,Walosczyk,16/12/1990,M,90121624113,Wrocław,ul. Zacisze 6/38,False,48567916007,swalosc@wp.pl
1515,1516,Patryk,Jabłoński,10/09/1977,M,77091087735,Wrocław,ul. Krótka 6/29,False,48631621442,p.jablonski1977@interia.pl
1516,1517,Dominik,Bździach,13/06/1996,M,96061339137,Słupsk,ul. Stawowa 5/1,False,48593450557,d-bzdzi1996@wp.pl
1517,1518,Marzena,Harvanko,14/02/2008,F,08221448506,Wrocław,ul. Podgórna 46,False,48285544943,harva_6831@gmail.com


In [57]:
np.sum(clients_df['karta_lojalnościowa'])

69

# auta